**Configure Devices**

In [1]:
import sys
import tensorflow as tf
sys.path.append('./rcwa_tf/src/')
import tf_utils

In [2]:
# Limit GPU memory growth
tf_utils.config_gpu_memory_usage()

# Choose the device to run on
use_gpu = True
tfDevice = '/job:localhost/replica:0/task:0/device:GPU:1' if use_gpu else '/CPU:0'
print('Executing on device ' + tfDevice + ' ...')

# Measure GPU memory usage
gpu_memory_init = tf_utils.gpu_memory_info()

 
2 Physical GPUs
Executing on device /job:localhost/replica:0/task:0/device:GPU:1 ...


**Dependencies**

In [3]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import pdb
import solver
import solver_metasurface
import rcwa_utils
import tensor_utils

**Definition of a Keras Layer with no Inputs** \
Used to hold height representation of the metasurface, which is treated as a trainable parameter.

In [5]:
class NoInput(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Simple, self).__init__(**kwargs)

    def build(self, input_shapes):
        self.kernel = self.add_weight(name='kernel', shape=self.output_dim, initializer='uniform', trainable=True)
        super(Simple, self).build(input_shapes)  

    def call(self, inputs):
        return self.kernel

    def compute_output_shape(self):
        return self.output_dim

**Metasurface Optimization TensorFlow Model Definition**

In [ ]:
class COPILOTMetasurfaceModel(Model):
    def __init__(self, user_params):
        super(MyModel, self).__init__()
        
        # Initialize and populate dictionary of solver parameters, based on the
        # dictionary of user-provided parameters.
        params = solver.initialize_params(wavelengths=user_params['wavelengths'],
                                          thetas=user_params['thetas'],
                                          phis=user_params['phis'],
                                          pte=user_params['pte'],
                                          ptm=user_params['ptm'],
                                          pixelsX=user_params['pixelsX'],
                                          pixelsY=user_params['pixelsY'],
                                          erd=user_params['erd'],
                                          ers=user_params['ers'],
                                          PQ=user_params['PQ'],
                                          Lx=user_params['Lx'],
                                          Ly=user_params['Ly'],
                                          L=user_params['L'],
                                          Nx=user_params['Nx'],
                                          eps_min=user_params['eps_min'],
                                          eps_max=user_params['eps_max'])

        # Merge with the user-provided parameter dictionary.
        params['N'] = user_params['N']
        params['w_l1'] = user_params['w_l1']
        params['w_l2'] = user_params['w_l2']
        params['thresholding_enabled'] = user_params['thresholding_enabled']
        params['sigmoid_coeff'] = user_params['sigmoid_coeff']
        params['sigmoid_update'] = user_params['sigmoid_update']
        params['learning_rate'] = user_params['learning_rate']
        params['focal_spot_radius'] = user_params['focal_spot_radius']
        params['random_init'] = user_params['random_init']
        params['debug'] = user_params['debug']

        # Define the free-space propagator and input field distribution for the metasurface.
        params['f'] = user_params['f'] * 1E-9
        params['upsample'] = user_params['upsample']
        params['propagator'] = solver.make_propagator(params, params['f'])
        params['input'] = solver.define_input_fields(params)
        
        # Save the dictionary of parameters as a class member.
        self.params = params
        
        # Simple layer (with no inputs) which holds lens parameters treated as trainable weights.
        self.lens_layer = NoInput(params['pixelsX'] * params['pixelsY'])

    def call(self, lam):
        '''
        Definition of the model's forward pass.
        Converts the given height representation of the metasurface to a stacked representation,
        which is then passed to the rcwa_tf solver. The resulting focal plane intensity pattern
        is used to generate a loss value, which is returned.
        Args:
            h: A `tf.Tensor` of shape `(pixelsX, pixelsY)` and type float containing heights of each pixel.
            
            params: A `dict` containing simulation and optimization settings.
            
        Returns:
            loss: Float loss value.
        '''
        
        # Get current height representation of the metasurface.
        h = self.lens_layer(None)
        
        # Reshape into a 2d height representation of the metasurface.
        h = tf.reshape(h, [params['pixelsX'],params['pixelsY']])
        
        # Generate permittivity and permeability distributions.
        ER_t, UR_t = solver_metasurface.generate_copilot_metasurface(h, params)

        # Simulate the system.
        outputs = solver.simulate(ER_t, UR_t, params)

        # Propagate the resulting fields to the focal plane.
        field = outputs['ty'][:, :, :, np.prod(params['PQ']) // 2, 0]
        focal_plane = solver.propagate(params['input'] * field, params['propagator'], params['upsample'])
        
        return focal_plane

**Loss Function Definition**

In [ ]:
def focal_spot(h, params):

    # Generate permittivity and permeability distributions.
    ER_t, UR_t = solver_metasurface.generate_copilot_metasurface(h, params)

    # Simulate the system.
    outputs = solver.simulate(ER_t, UR_t, params)

    # First loss term: maximize sum of electric field magnitude within some radius of the desired focal point.
    r = params['focal_spot_radius']
    field = outputs['ty'][:, :, :, np.prod(params['PQ']) // 2, 0]
    focal_plane = solver.propagate(params['input'] * field, params['propagator'], params['upsample'])
    index = (params['pixelsX'] * params['upsample']) // 2
    l1 = tf.math.reduce_sum(tf.abs(focal_plane[0, index-r:index+r, index-r:index+r]))

    # Second loss term: minimize sum of electric field magnitude elsewhere.
    l2 = tf.math.reduce_sum(tf.abs(focal_plane[0, :, :])) - l1

    # Final loss: (negative) field intensity at focal point + field intensity elsewhere.
    return -params['w_l1']*l1 + params['w_l2']*l2

**Optimize Device Function Definition**

In [ ]:
def optimize_device(user_params):
    
    # Initialize and populate dictionary of solver parameters, based on the
    # dictionary of user-provided parameters.
    params = solver.initialize_params(wavelengths=user_params['wavelengths'],
                                      thetas=user_params['thetas'],
                                      phis=user_params['phis'],
                                      pte=user_params['pte'],
                                      ptm=user_params['ptm'],
                                      pixelsX=user_params['pixelsX'],
                                      pixelsY=user_params['pixelsY'],
                                      erd=user_params['erd'],
                                      ers=user_params['ers'],
                                      PQ=user_params['PQ'],
                                      Lx=user_params['Lx'],
                                      Ly=user_params['Ly'],
                                      L=user_params['L'],
                                      Nx=user_params['Nx'],
                                      eps_min=user_params['eps_min'],
                                      eps_max=user_params['eps_max'])
    
    # Merge with the user-provided parameter dictionary.
    params['N'] = user_params['N']
    params['w_l1'] = user_params['w_l1']
    params['w_l2'] = user_params['w_l2']
    params['thresholding_enabled'] = user_params['thresholding_enabled']
    params['sigmoid_coeff'] = user_params['sigmoid_coeff']
    params['sigmoid_update'] = user_params['sigmoid_update']
    params['learning_rate'] = user_params['learning_rate']
    params['focal_spot_radius'] = user_params['focal_spot_radius']
    params['random_init'] = user_params['random_init']
    params['debug'] = user_params['debug']

    # Define the free-space propagator and input field distribution for the metasurface.
    params['f'] = user_params['f'] * 1E-9
    params['upsample'] = user_params['upsample']
    params['propagator'] = solver.make_propagator(params, params['f'])
    params['input'] = solver.define_input_fields(params)
    
    # Get initial heights of the metasurface.
    h_initial = np.zeros((params['pixelsX'],params['pixelsY']))
    h = tf.Variable(h_initial, dtype=tf.float32)
    
    # Define an optimizer and data to be stored.
    opt = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    loss = np.zeros(params['N'] + 1)
    
    # Optimize.
    print('Optimizing... Iteration ', end="")
    
    i = 0
    while i < params['N']:
        print(str(i) + ', ', end="")
        
        # Calculate gradients.
        with tf.GradientTape() as tape:
            l = focal_spot(h, params)
            grads = tape.gradient(l, [h])
        
        # Modify gradients to adhere to problem constraints.
        #constrained_grads = [constrain_gradients(g, device) for g in grads]
        
        # Apply gradients to variables.
        opt.apply_gradients(zip(grads, [h]))
        
        loss[i] = l.numpy()
        
        # Anneal sigmoid coefficient.
        if params['thresholding_enabled']:
            params['sigmoid_coeff'] += params['sigmoid_update']
            
        # Increment loop counter.
        i += 1
    
    print('Maximum iterations reached.')
        
    # Round off to a final, admissable, solution.
    # Do a final range clip.
    h = tf.clip_by_value(h, clip_value_min = 0, clip_value_max = params['Nlay']-1)
    
    # Round heights to nearest integer.
    h = tf.math.round(h)
    
    # Get final loss.
    loss[i] = focal_spot(h, params).numpy()
    print('Final Loss: ' + str(loss[i]))
    
    return h, loss, params

**Initialize Parameters**

In [ ]:
with tf.device(tfDevice):
    
    # Initialize dict of user-configurable parameters.
    user_params = {}
    
    # Source parameters.
    user_params['wavelengths'] = [120000.0]
    user_params['thetas'] = [0.0]
    user_params['phis'] = [0.0]
    user_params['pte'] = [1.0]
    user_params['ptm'] = [0.0]
    
    # Device parmeters.
    user_params['pixelsX'] = 15
    user_params['pixelsY'] = user_params['pixelsX']
    user_params['erd'] = 12.04
    user_params['ers'] = user_params['erd']
    user_params['eps_min'] = 1.0
    user_params['eps_max'] = user_params['erd']
    user_params['Nlay'] = 6
    user_params['L'] = [50000.0] * user_params['Nlay']
    user_params['Lx'] = 20000.0
    user_params['Ly'] = user_params['Lx']
    user_params['f'] = 1.0 # Focal distance (nm)
    
    # Solver parameters.
    user_params['PQ'] = [5,5]
    user_params['Nx'] = 16
    user_params['Ny'] = user_params['Nx']
    user_params['upsample'] = 11
    
    # Problem parameters.
    user_params['N'] = 100
    user_params['w_l1'] = 1.0
    user_params['w_l2'] = 0.0
    user_params['thresholding_enabled'] = True
    user_params['sigmoid_coeff'] = 1.0
    user_params['sigmoid_update'] = 40.0 / user_params['N']
    user_params['learning_rate'] = 4E-1
    user_params['focal_spot_radius'] = 10
    user_params['random_init'] = False
    user_params['debug'] = False

**Optimize**

In [ ]:
with tf.device(tfDevice):
    
    h, loss, params = optimize_device(params)

**Display Learning Curve**

In [ ]:
with tf.device(tfDevice):
    
    plt.plot(loss)
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.show()

**Display Resulting Intensity on the Focal Plane**

In [ ]:
with tf.device(tfDevice):
    
    ER_t, UR_t = solver_metasurface.generate_copilot_metasurface(h, params)
    outputs = solver.simulate(ER_t, UR_t, params)
    field = outputs['ty'][:, :, :, np.prod(params['PQ']) // 2, 0]
    focal_plane = solver.propagate(params['input'] * field, params['propagator'], params['upsample'])
    plt.imshow(tf.abs(focal_plane[0, :, :]) ** 2)
    plt.colorbar()

**Display Metasurface**

In [ ]:
with tf.device(tfDevice):
    
    solver_metasurface.display_metasurface(ER_t, params)
    print(h)

**Evaluate Metasurface**

In [ ]:
with tf.device(tfDevice):
    
    eval = solver_metasurface.evaluate_solution(focal_plane, params)
    print('Evaluation Score of the Metasurface: ' + str(eval))

**Check GPU Memory Utilization**

In [ ]:
gpu_memory_final = tf_utils.gpu_memory_info()
gpu_memory_used = [gpu_memory_final[1][0] - gpu_memory_init[1][0], gpu_memory_final[1][1] - gpu_memory_init[1][1]]
print('Memory used on each GPU(MiB): ' + str(gpu_memory_used))